## journal-ref列の特徴量を作成する

In [2]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm.notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

data_path = Path.Path("../data")
img_path = Path.Path("../img")
result_path = Path.Path("../result")

prefix = 'ana005'

## データ読み込み

In [3]:
train_df = pd.read_pickle(data_path / 'train_df.pkl')

train_df

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[[{'version': 'v1', 'created': 'Wed, 10 Feb 19...",2008-11-26,"[[Hayes, Mark E., , University College London]...",1,NaN
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[[{'version': 'v1', 'created': 'Thu, 27 Mar 20...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[[{'version': 'v1', 'created': 'Thu, 22 May 20...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0
3,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[[{'version': 'v1', 'created': 'Tue, 3 Jul 201...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",6,8.0
4,1905.05921,Juanjuan Gu,Juanjuan Gu and Yun Jing,A Modified Mixed Domain Method for Modeling Ac...,None,None,10.1121/10.0001454,None,physics.med-ph physics.comp-ph,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, phase correction and amplitud...","[[{'version': 'v1', 'created': 'Wed, 15 May 20...",2020-07-15,"[[Gu, Juanjuan, ], [Jing, Yun, ]]",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851519,1708.06097,Haim Diamant,Rony Granek and Haim Diamant,Membrane undulations in a structured fluid: Un...,10 pages,"Eur. Phys. J. E 41, 1 (2018)",10.1140/epje/i2018-11607-x,None,cond-mat.soft,http://arxiv.org/licenses/nonexclusive-distrib...,The dynamics of membrane undulations inside ...,"[[{'version': 'v1', 'created': 'Mon, 21 Aug 20...",2018-02-20,"[[Granek, Rony, ], [Diamant, Haim, ]]",0,NaN
851520,1904.07627,Chonglong Liu,"C. L. Liu, Xiao-Dong Yu, D. M. Tong",Flag Additivity in Quantum Resource Theories,6 pages,"Phys. Rev. A 99, 042322(2019)",10.1103/PhysRevA.99.042322,None,quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Quantum resource theories offer a powerful f...,"[[{'version': 'v1', 'created': 'Tue, 16 Apr 20...",2019-04-17,"[[Liu, C. L., ], [Yu, Xiao-Dong, ], [Tong, D. ...",2,NaN
851521,1507.01140,Othman Benomar,"O. Benomar, M. Takata, H. Shibahashi, T. Ceill...",Nearly-uniform internal rotation of solar-like...,Accepted to MNRAS,None,10.1093/mnras/stv1493,None,astro-ph.SR,http://arxiv.org/licenses/nonexclusive-distrib...,The rotation rates in the deep interior and ...,"[[{'version': 'v1', 'created': 'Sat, 4 Jul 201...",2015-08-06,"[[Benomar, O., ], [Takata, M., ], [Shibahashi,...",42,NaN
851522,2006.03674,Sukriti Manna,"Troy D Loeffler, Sukriti Manna, Tarak K Patra,...",Active Learning A Neural Network Model For Gol...,None,None,10.1002/cctc.202000774,None,physics.comp-ph cond-mat.mtrl-sci,http://arxiv.org/licenses/nonexclusive-distrib...,Small metal clusters are of fundamental scie...,"[[{'version': 'v1', 'created': 'Fri, 5 Jun 202..."

## 特徴量作成
> journal-ref列から取得できる要素を精査

In [6]:
# 上から100行出してみる
list(train_df['journal-ref'][:100])

['J.Phys.G25:1477-1493,1999',
 None,
 'JCAP10(2014)035',
 'Nat Commun 10, 3364 (2019)',
 None,
 None,
 'Astrophys.J. 534 (2000) 581-586',
 'Eur.Phys.J.C20:507-515,2001',
 'J. Quant. Spectros. Radiat. Transf., 2015, 152: 114-126',
 None,
 'Phys.Lett.B451:98-104,1999',
 'JHEP 0111:034,2001',
 'Chaos 23, 013112 (2013) Research Highlights',
 'Phys. Rev. E 92, 012118 (2015)',
 'Int.J.Mod.Phys. A20 (2005) 6959-6961',
 'Phys. Rev. Lett. 120, 141601 (2018)',
 'Physical Review E, V.70, 066618 (2004)',
 'AIP Conf.Proc.915:642-645,2007',
 None,
 None,
 'AIP Conf.Proc.1103:571-581,2009',
 'Lect.Notes Phys. 506 (1999) 211',
 None,
 None,
 'ApJ, 897, 79, 2020',
 'International Journal of Remote Sensing, Taylor \\& Francis, 2010,\n  31 (22), pp.5895-5920',
 'Phys. Plasmas 22, 042109 (2015)',
 'Phys. Rev. B 80, 214414 (2009)',
 None,
 'Phys. Rev. D 94, 034018 (2016)',
 'Nonlinearity Vol. 19, p. 1233-1252, 2006',
 'JHEP 0709:102,2007',
 None,
 '1999, ApJS, 122, 221',
 'Mod.Phys.Lett. A15 (2000) 1767-17

> 掲載紙の出版年が抽出できそう

In [37]:
import re

def extract_year(input_str):
    
    if input_str is None: return np.nan

    # 括弧の中に4桁の数字があるもの、末尾に4桁の数字があるものを取得
    find_list = re.findall(r'\((\d{4})\)', input_str) + re.findall(r'[^\d](\d{4})$', input_str)

    # どちらかもしくは両方が取得できた場合、括弧内4桁数字を優先に返却、どちらもなかった場合nanを返却
    return int(find_list[0]) if len(find_list) > 0 else np.nan

In [38]:
%%time

# 特徴量DFを用意
j_ref_x_df = pd.DataFrame()

# 掲載紙の出版年を作成
j_ref_x_df['ref-year'] = train_df['journal-ref'].apply(lambda x: extract_year(x)).astype('Int64')

# 1980 ~ 2022 まで限定
j_ref_x_df['ref-year'] = j_ref_x_df['ref-year'].mask((j_ref_x_df['ref-year'] < 1980) | (j_ref_x_df['ref-year'] > 2022), np.nan)

j_ref_x_df

Wall time: 2.54 s


,ref-year
0,1999
1,<NA>
2,2014
3,2019
4,<NA>
...,...
851519,2018
851520,2019
851521,<NA>
851522,<NA>


In [41]:
j_ref_x_df['ref-year'].value_counts(dropna=False)

NaN     336532
2019     31662
2018     30770
2017     29290
2016     28625
2015     25710
2014     24920
2009     24008
2010     23812
2011     22616
2013     22371
2007     21979
2006     21629
2008     21541
2012     21303
2005     20180
2020     19786
2004     18992
2003     17285
2002     15056
2001     13238
2000     11954
1999     10415
1998      9551
1997      7636
1996      6653
1995      5603
1994      4319
1993      2821
1992      1002
1991        90
1990        35
2021        34
1988        23
1986        16
1987        14
1982        11
1984        11
1989        10
1983         7
1985         5
1980         3
1981         3
2022         3
Name: ref-year, dtype: Int64

> 出版年が取れないデータは、扱いが難しい上に多そう(平均値とかで穴埋め？)